In [5]:
!pip install nltk
!pip install mysql-connector-python


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# Summarizer

In [6]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk


nltk.download('stopwords')
nltk.download("punkt")
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\giuli\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\giuli\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\giuli\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [7]:
def getsentenceValue(freqTable, text_to_summarize):
  sentenceValue = dict()
  for sentence in sent_tokenize(text_to_summarize):
    for word, freq in freqTable.items():
      if word in sentence.lower():
        if sentence in sentenceValue: sentenceValue[sentence] += freq
        else: sentenceValue[sentence] = freq
  return sentenceValue

def getSumValues(sentenceValue):
  sumValues = 0
  for sentence in sentenceValue:
    sumValues += sentenceValue[sentence]

  average = int(sumValues / len(sentenceValue))
  return average



In [8]:
def summaryManager(text_to_summarize):
  freqTable = dict()
  summary = ''

  for word in word_tokenize(text_to_summarize):
    word = word.lower()
    if word in set(stopwords.words("english")): continue
    if word in freqTable: freqTable[word] += 1
    else: freqTable[word] = 1



  sentenceValue = getsentenceValue(freqTable, text_to_summarize)
  print(f"sentenceValue: {sentenceValue} -- {sentenceValue == {}}")
  if sentenceValue == {}: return text_to_summarize

  average = getSumValues(sentenceValue)
  # print(average)


  for sentence in sent_tokenize(text_to_summarize):
    if (sentence in sentenceValue) and (sentenceValue[sentence] > (1.2 * average)):
      summary += " " + sentence

  return summary

In [9]:
text = """L'avvento dell'intelligenza artificiale (IA) ha rivoluzionato numerosi settori, dalla medicina all'industria automobilistica. Ma le sue radici affondano in un passato più remoto di quanto si possa immaginare. Uno dei primi esempi di come la macchina potesse assistere l'uomo nella risoluzione di problemi complessi risale alla Seconda Guerra Mondiale.

La macchina Enigma, utilizzata dai nazisti per criptare le comunicazioni, rappresentava una delle sfide più ardue per gli alleati. La sua complessità, con milioni di possibili configurazioni, rendeva la decifrazione un'impresa titanica. Tuttavia, un gruppo di crittoanalisti, guidati dall'eccentrico Alan Turing, riuscì nell'impresa di costruire una macchina, chiamata Colossus, in grado di decifrare i messaggi nemici.

Colossus, considerata il primo computer elettronico programmabile, era un'anteprima di ciò che sarebbe diventata l'IA. La sua capacità di eseguire calcoli complessi a una velocità inimmaginabile per l'epoca permise di abbattere il muro di cifratura nazista, accelerando così la fine della guerra.

Oggi, le tecniche di apprendimento automatico, un sottocampo dell'IA, sono utilizzate per affrontare problemi simili, ma su scala molto più vasta. Gli algoritmi di machine learning sono in grado di individuare pattern complessi in enormi quantità di dati, una capacità che si rivela fondamentale in settori come la finanza, la sicurezza informatica e la ricerca scientifica.

Tuttavia, mentre Colossus era progettata per un compito specifico, l'IA moderna è molto più versatile. Le reti neurali artificiali, ispirate al funzionamento del cervello umano, sono in grado di apprendere da esempi e generalizzare a nuovi dati. Questa capacità ha aperto le porte a una miriade di applicazioni, dalla traduzione automatica alla generazione di testi creativi.

È interessante notare come l'obiettivo iniziale di decifrare un codice segreto si sia evoluto in una disciplina che mira a imitare l'intelligenza umana. L'eredità di Alan Turing e di Colossus continua a ispirare i ricercatori di oggi, che lavorano per sviluppare macchine sempre più intelligenti e in grado di risolvere problemi sempre più complessi."""




print( summaryManager(text) )

sentenceValue: {"L'avvento dell'intelligenza artificiale (IA) ha rivoluzionato numerosi settori, dalla medicina all'industria automobilistica.": 109, 'Ma le sue radici affondano in un passato più remoto di quanto si possa immaginare.': 71, "Uno dei primi esempi di come la macchina potesse assistere l'uomo nella risoluzione di problemi complessi risale alla Seconda Guerra Mondiale.": 98, 'La macchina Enigma, utilizzata dai nazisti per criptare le comunicazioni, rappresentava una delle sfide più ardue per gli alleati.': 102, "La sua complessità, con milioni di possibili configurazioni, rendeva la decifrazione un'impresa titanica.": 107, "Tuttavia, un gruppo di crittoanalisti, guidati dall'eccentrico Alan Turing, riuscì nell'impresa di costruire una macchina, chiamata Colossus, in grado di decifrare i messaggi nemici.": 122, "Colossus, considerata il primo computer elettronico programmabile, era un'anteprima di ciò che sarebbe diventata l'IA.": 106, "La sua capacità di eseguire calcoli co

# DB manager

In [12]:
import mysql.connector

# Connection to DB
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  port="3307",
  password="root",
  database="bookshop_db"
)


# Get list of book to create summary
mycursor = mydb.cursor()
mycursor.execute("SELECT idBook, Description FROM book WHERE reassume IS NULL;")
myresult = mycursor.fetchall()



for i in range(len(myresult)):
  try: summary = summaryManager(myresult[i][1])
  except: continue
  if summary == "": continue 
  
  # Insert SUMMARY
  sql = 'UPDATE book SET reassume="' + summary + '" WHERE idBook = ' + str(myresult[i][0]) + ' ;'
  mycursor.execute(sql)
  mydb.commit()


mydb.close()

0
sentenceValue: {'From Richard Lyntton, an explosive international thriller featuring British journalist, veteran tank commander, and trained killer (retired), Jack Steele.': 37, 'An unimaginable nuclear disaster.': 8, 'Thousands of lives are at stake.': 8, 'One man can prevent it ... as a global conspiracy plots his assassination.': 13, 'Set in London, Moscow, Vladivostok, and Pyongyang buckle up for this gripping, action-packed thriller of international intrigue, espionage, suspense, and deceit.': 32} -- False


DatabaseError: 1205 (HY000): Lock wait timeout exceeded; try restarting transaction

## Clear DB

In [21]:
import mysql.connector

# Connection to DB
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  port="3307",
  password="root",
  database="bookshop_db"
)


# Get list of book to create summary
mycursor = mydb.cursor()
mycursor.execute("SELECT idBook, Description FROM book;")
myresult = mycursor.fetchall()




for i in range(len(myresult)):
    sql = "UPDATE book SET reassume=NULL WHERE idBook = " + str(myresult[i][0]) + ";"

    print(f"id {myresult[i][0]}   --- sql: {sql}")
    mycursor.execute(sql)
    mydb.commit()


mydb.close()

id 14   --- sql: UPDATE book SET reassume=NULL WHERE idBook = 14;
id 16   --- sql: UPDATE book SET reassume=NULL WHERE idBook = 16;
id 17   --- sql: UPDATE book SET reassume=NULL WHERE idBook = 17;
id 23   --- sql: UPDATE book SET reassume=NULL WHERE idBook = 23;
id 24   --- sql: UPDATE book SET reassume=NULL WHERE idBook = 24;
id 25   --- sql: UPDATE book SET reassume=NULL WHERE idBook = 25;
id 26   --- sql: UPDATE book SET reassume=NULL WHERE idBook = 26;
id 27   --- sql: UPDATE book SET reassume=NULL WHERE idBook = 27;
id 28   --- sql: UPDATE book SET reassume=NULL WHERE idBook = 28;
id 29   --- sql: UPDATE book SET reassume=NULL WHERE idBook = 29;
id 30   --- sql: UPDATE book SET reassume=NULL WHERE idBook = 30;
id 31   --- sql: UPDATE book SET reassume=NULL WHERE idBook = 31;
id 35   --- sql: UPDATE book SET reassume=NULL WHERE idBook = 35;
